# 🧠 MCP 1.20 Demo + LangChain Integration
**Date:** October 31, 2025

This notebook demonstrates a working Model Context Protocol (MCP) Server + Client using the Anthropic SDK (v1.20 series), and an optional LangChain integration with GPT‑5.

Ensure you have `mcp`, `langchain‑mcp`, and `langchain‑openai` installed:
```bash
pip install -U mcp langchain-mcp langchain-openai
```

In [ ]:
%pip install -U langchain-mcp

In [7]:
%pip show langchain-mcp

Name: langchain-mcp
Version: 0.2.1
Summary: Model Context Protocol tool calling support for LangChain
Home-page: 
Author: 
Author-email: Andrew Wason <rectalogic@rectalogic.com>
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: langchain-core, mcp, pydantic, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## 🧩 Cell 1 — Define and Run MCP Server
Defines two tools: `add` and `message_of_the_day`.

**Run this cell once**; it will wait for a client connection (stdio).

In [39]:
# Basic GPT‑5 text generation using the Responses API
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

resp = client.responses.create(
    model="gpt-5" ,  # or the exact variant you have access to, e.g., "gpt-5-large"
    input="Explain the Model Context Protocol in one short paragraph for a banking audience."
)
print(resp.output_text)

The Model Context Protocol (MCP) is a standard way to let AI systems securely “plug into” enterprise data and tools at runtime, without copying sensitive banking data into the model or retraining it. MCP exposes approved tools and read-only resources through a governed interface, so an AI assistant can, for example, fetch a customer profile, run a risk check, or query a ledger with least-privilege access and full audit trails. Because the protocol is model- and vendor-agnostic, banks can reuse the same controls across different AI providers and deployment modes (cloud or on‑prem). Policies, permissions, and human approvals can be enforced centrally, and access can be revoked instantly. The result is faster AI integration with core systems while supporting compliance, data minimization, and operational risk controls.


In [1]:
%%writefile mcp_server.py

from mcp.server.fastmcp import FastMCP

mcp = FastMCP("DemoServer")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

@mcp.tool()
def message_of_the_day() -> str:
    """Return a motivational message."""
    return "Stay curious. Build boldly."

if __name__ == "__main__":
    # FastMCP.run() is synchronous in v1.20+
    mcp.run()

Overwriting mcp_server.py


## 🧪 Cell 2 — Connect Client and Parse Results
Launches the server automatically via stdio, lists available tools, and calls them.

In [2]:
%%writefile mcp_client_demo.py

import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def main():
    params = StdioServerParameters(command="python", args=["mcp_server.py"])

    async with stdio_client(params) as (r, w):
        async with ClientSession(r, w) as session:
            await session.initialize()
            tools = await session.list_tools()
            tool_names = [t[0] for t in tools]
            print("Available tools:", tool_names)

            result = await session.call_tool("add", {"a": 7, "b": 8})
            print("add(7,8) =", result.structuredContent.get("result"))

            motd = await session.call_tool("message_of_the_day", {})
            print("message_of_the_day() =", motd.structuredContent.get("result"))

asyncio.run(main())


Writing mcp_client_demo.py


In [3]:
!python mcp_client_demo.py

[10/31/25 20:41:34] INFO     Processing request of type            ]8;id=592778;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=203149;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             ListToolsRequest                                   
Available tools: ['meta', 'nextCursor', 'tools']
                    INFO     Processing request of type            ]8;id=643674;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=785493;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             CallToolRequest                                    
add(7,8) = 15
                    INFO     Processing request of type            ]8;id=921498;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=633470

## 🤖 Cell 3 — LangChain + GPT‑5 Integration
Demonstrates using LangChain to allow GPT‑5 to call MCP tools automatically.

In [42]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

resp = client.responses.create(model="gpt-5", input="Hello GPT-5, just testing connectivity.")
print(resp.output_text)

You’re connected—I’m here and ready. How can I help today?


In [48]:
%%writefile langchain_mcp_integration1.py

import asyncio
import os
import warnings

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp import MCPToolkit
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent  # modern agent type

warnings.filterwarnings("ignore", category=UserWarning)

# Optional: set your key here for testing
# os.environ["OPENAI_API_KEY"] = "sk-your-key"

async def run_agent():
    params = StdioServerParameters(command="python", args=["mcp_server.py"])

    async with stdio_client(params) as (r, w):
        async with ClientSession(r, w) as session:
            await session.initialize()

            # Build and initialize MCP toolkit
            toolkit = MCPToolkit(session=session)
            await toolkit.initialize()
            tools = toolkit.get_tools()
            print("✅ Loaded tools:", [t.name for t in tools])

            # Create GPT-5 LLM with timeout and verbosity
            llm = ChatOpenAI(model="gpt-5", temperature=0, request_timeout=30, verbose=True)

            # Create modern LangGraph agent that supports multi-input tools
            agent = create_react_agent(llm, tools)

            print("🤖 Running agent query ...\n")
            try:
                # Use async .ainvoke() (non-blocking)
                result = await agent.ainvoke(
                    {"messages": [("user", "Add 25 and 30 using the MCP tool, then show the message of the day.")]}
                )

                # Extract last message content safely
                last_message = result["messages"][-1].content
                print("\n💬 Agent Response:\n", last_message)

            except Exception as e:
                print(f"⚠️ Agent execution failed: {e}")

if __name__ == "__main__":
    asyncio.run(run_agent())


Writing langchain_mcp_integration1.py


In [49]:
!python langchain_mcp_integration1.py

[10/31/25 22:40:39] INFO     Processing request of type            ]8;id=907481;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=750516;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             ListToolsRequest                                   
✅ Loaded tools: ['add', 'message_of_the_day']
🤖 Running agent query ...

[10/31/25 22:40:45] INFO     Processing request of type            ]8;id=688997;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=576236;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             CallToolRequest                                    
                    INFO     Processing request of type            ]8;id=174636;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8